In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import os

In [2]:
max_dataset_size = 6000000
waveform_length = 72;
nb_of_elements = 100000
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
root_folder = os.path.dirname(os.getcwd())
path_to_recording= path.join(root_folder,'data/recording_datasets/datasets_1.npy');
path_to_ground_truth_data = path.join(root_folder,'data/recording_datasets/ground_truth_data_multiunit_1.npy');
path_to_train_data = path.join(root_folder,'data/train_data.npy');
path_to_train_labels = path.join(root_folder,'data/train_labels.npy');
path_to_noise_data = path.join(root_folder,'data/noise_data.npy');
path_to_mean_std = path.join(root_folder,'data/mean_std.npy');
sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [4]:
noise_indices = cnn.GetNoiseIndices(path_to_recording, path_to_ground_truth_data, waveform_length, nb_of_elements);



In [ ]:
noise_class = torch.zeros(1, noise_indices.nelement(), dtype= torch.int);
noise_data = torch.cat((noise_indices, noise_class), 0);


In [ ]:
np.save(path_to_noise_data, noise_data.numpy());

In [ ]:
dataset_spikes = cnn.GenerateDataset(path_to_recording, path_to_ground_truth_data, waveform_length, max_dataset_size);


========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  4
shift_indexes:  tensor([-18, -14, -10,  -6,  -2,   2,   6,  10,  14,  18], dtype=torch.int32)
snr_ratio:  None
flip_data_horz:  0
flip_data_vert:  0


/home/vtpc/Documents/Alvils/spike-sorting/custom_resnet/CustomResnet.py:291: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  temp = temp.new_tensor(data);


temp_dataset_len_after:  509580
temp_dataset_len_after:  1019160
dataset len:  1019160
========== 2 generation ==========
temp_dataset_len:  0
shift_from:  -14
shift_to:  12
shift_step:  2
shift_indexes:  tensor([-14, -12, -10,  -8,  -6,  -4,  -2,   0,   2,   4,   6,   8,  10],
       dtype=torch.int32)
snr_ratio:  7.363793755588508
flip_data_horz:  1
flip_data_vert:  0
72000000
temp_dataset_len_after:  662454
72000000
temp_dataset_len_after:  1324908
dataset len:  2344068
========== 3 generation ==========
temp_dataset_len:  0
shift_from:  -1
shift_to:  6
shift_step:  4
shift_indexes:  tensor([-1,  3], dtype=torch.int32)
snr_ratio:  15.733555107696612
flip_data_horz:  1
flip_data_vert:  1
72000000
temp_dataset_len_after:  101916
72000000
temp_dataset_len_after:  203832
dataset len:  2547900
========== 4 generation ==========
temp_dataset_len:  0
shift_from:  -7
shift_to:  12
shift_step:  4
shift_indexes:  tensor([-7, -3,  1,  5,  9], dtype=torch.int32)
snr_ratio:  14.165114860629286
f

In [ ]:
dataset_noise = cnn.GenerateDataset(path_to_recording, path_to_noise_data, waveform_length, max_dataset_size);


In [ ]:
dataset_size =  dataset_spikes.__len__() if dataset_spikes.__len__() < dataset_noise.__len__() else  dataset_noise.__len__()

In [ ]:
# convers to np array
np_data_spikes = np.zeros((dataset_size, 1, waveform_length));
for i, (data, target) in enumerate(dataset_spikes):
  np_data_spikes[i, :] = data.numpy();
  if(i + 1 == dataset_size):
    break;
    
del dataset_spikes

# removes spikes that have mutiple spikes in waveform
spike_argmax = np.argmax(np_data_spikes, 2)
over_treshold = waveform_length // 2 + waveform_length // 4
under_treshold = waveform_length // 2 - waveform_length // 4
is_valid_train_data = ((spike_argmax >= under_treshold) & (spike_argmax <= over_treshold)).ravel()
np_data_spikes = np_data_spikes[is_valid_train_data, :, :]
dataset_size = np_data_spikes.shape[0]
np_classes_spikes = np.ones(dataset_size)


np_data_noise = np.zeros((dataset_size, 1, waveform_length));
np_classes_noise = np.zeros(dataset_size)
for i, (data, target) in enumerate(dataset_noise):
  np_data_noise[i, :] = data.numpy();
  if(i + 1 == dataset_size):
    break;
    
del dataset_noise

    


In [ ]:
np_data_noise.shape

In [ ]:
dataset = np.concatenate((np_data_spikes, np_data_noise), 0);

In [ ]:
labels = np.concatenate((np_classes_spikes, np_classes_noise), 0); 

In [ ]:
np.save(path_to_train_data, dataset)
np.save(path_to_train_labels, labels)


In [ ]:
mean = (np.mean(np_data_spikes))
std = (np.std(np_data_spikes))
np.save(path_to_mean_std, [mean, std])

print(mean)
print(std)

In [ ]:
import matplotlib.pyplot as plt
rnd = np.random.randint(0, dataset_size);
plt.plot(np_data_spikes[rnd, 0, :])
np.argmax(np_data_spikes[rnd, 0, :])